# Electron density topology analysis using TOPOND

TOPOND is a program contained inside CRYSTAL which performs the analysis of the electron density, $\rho (\mathbf{r})$, and related quantities, under the Quantum Theory of Atoms in Molecules and Crystals (QTAIM). 

In this notebook, different tools that facilitate the analysis and visualization of the data generated by TOPOND, which are available in the CRYSTALClear package, are shown. Pregenerated output files are provided.

---

## Tools for TRHO calculations

The TRHO option performs an analysis of the Topology of $\rho$. It provides information on the Critical Points (CP) of $\rho$, which can be classified as Nuclear or Non-Nuclear Attractors (NNA), Bond CP (BCP), Ring CP (RCP) and Cage CP (CCP). At each located CP relevant quantities such as $\nabla \rho$ and $\nabla^2 \rho$ among others, are calculated and reported in the output. For each BCP, the NNA associated points are located tracking its Atomic Interaction Lines (AIL) or Bond Paths (BP), indicating also the geometrical and the BP distances between NNAs.

In [ ]:
# To be used while the functions are not in Release
!git clone https://github.com/crystaldevs/CRYSTALClear.git
!pip install ./CRYSTALClear

In [ ]:
# After included in Release
!pip install CRYSTALClear

Tools related to TOPOND require the generation of a Properties_output object:

In [ ]:
from CRYSTALClear.crystal_io import Properties_output

In [ ]:
urea_mol = Properties_output()

`read_topond_trho` takes as argument an output file of a TRHO run.

In [ ]:
# Urea molecule, using IAUTO=-2. CRY23
urea_mol.read_topond_trho("outputs/urea_mol_trho_cry23.outp")

After this, a table (`pandas` DataFrame) called `topo_df` with the information of every CP founded should be available under the corresponding object.


In [ ]:
# Check DF
urea_mol.topo_df

This table can be easily postprocessed or consulted to characterize or classify a given CP, as well as to performe statistical analysis.

In [ ]:
# Some general statistics of the DF
urea_mol.topo_df.describe()

In order to visualize the CP in a 3D representation, an external file can be produced using the `topond_viz_file` method. Using the package `ASE` for intermediate processing, it can produce files in common formats (such as `XYZ`, `PDB`, `CIF`) to be visualized using programs like Jmol, VMD or VESTA. The function also returns an `ASE Atoms` object, which allows for further processing using its associated functions.

In [ ]:
# Generate output file to visualize the CP.
# It is generated in the same location as the TOPOND output file.
ase_obj = urea_mol.topond_viz_file(cp_type='BCP',add_atoms=True,file_type='cif')

If only a basic visualization is required, the `py3Dmol` package can be used on a Jupyter Notebook. The following cell shows a use case for the CIF file generated in the previous cell:

In [ ]:
# Notebook visualization example, using generated file
# If not installed, uncomment following line:
#!pip install py3dmol

import py3Dmol
''' py3Dmol mouse controls:

Left click: Rotation
Ctrl + Left click: Translate
Right click: Zoom
'''

# Storing file lines into an array for py3Dmol usage
viz_file = 'outputs/urea_mol_trho_cry23_BCP.cif'
with open(viz_file) as in_file:
    system = "".join([x for x in in_file])

view = py3Dmol.view(width=600, height=400)

# Change accordingly the file type
view.addModel(system,'cif')

view.addUnitCell()

# Double representation needed to obtain ball-and-stick style
view.setStyle({"stick":{"radius":0.05}})
#view.addStyle({"sphere":{"radius":0.3}})

# To plot the CPs
view.setStyle({"elem":'X'},{"sphere":{"color":'violet', "radius":0.1}})

# To include Atom ID labels
view.addPropertyLabels('index',{"elem":'X'}, {"backgroundOpacity" : 0.5, "fontSize" : 12} )

#view.setBackgroundColor('0xeeeeee')
#view.zoomTo()

view.show()

### Extra: Electrostatic Potential Maps with py3Dmol

Note: For dense grids the visualization takes too much time.

In [ ]:
# Storing file lines of each cube file
rho_file = 'outputs/so2_rho.cube'
with open(rho_file) as in_file:
    rho_sys = "".join([x for x in in_file])

potel_file = 'outputs/so2_potel.cube'
with open(potel_file) as in_file:
    potel_sys = "".join([x for x in in_file])

view = py3Dmol.view(width=600, height=400)

view.addVolumetricData(rho_sys, "cube", {'isoval': 0.01, 'smoothness': 5, 'opacity':0.7, 'voldata': potel_sys, 'volformat': 'cube', 
      'volscheme': {'gradient':'rwb', 'min':-.05, 'max':.05}})

# To include the atoms
view.addModel(rho_sys,'cube')

# Double representation needed to obtain ball-and-stick style
view.setStyle({"stick":{"radius":0.1}})

#view.zoomTo()
#view.show()